# Search runs

In [ ]:
# importing necessary libraries
from mlflow_for_ml_dev.experiments.exp_utils import get_or_create_experiment

# additional libraries
import mlflow 
import random
from typing import Dict 

In [ ]:
experiment_name = "searching_for_runs"
tags = {"project_name":"UNDEFINED", "topic":"searching"}
experiment = get_or_create_experiment(experiment_name, tags)

## Generating Demo Runs

In [ ]:
def create_demo_run(run_name:str, experiment_id:str, run_tags:Dict[str,str]):
    """
    Create a run with the given name, experiment_id and tags.

    :param run_name: The name of the run
    :param experiment_id: The id of the experiment
    :param run_tags: The tags of the run
    :return: The id of the run
    """
    with mlflow.start_run(run_name=run_name, experiment_id=experiment_id, tags=run_tags) as run:
        mlflow.log_params({
            "param1": random.randint(0, 100),
            "param2": random.randint(0, 100),
            "param3": random.randint(0, 100)
        })	
        mlflow.log_metrics({
            "metric1": random.random(),
            "metric2": random.random(),
            "metric3": random.random()
        })

    return run.info.run_id

In [ ]:
for n in range(100):
    run_tags = {"run_type":random.choice(["experiment","optimization"]),"task":random.choice(["regression","classification"])}
    run_name = random.choice(["random_run", "important_run"])
    run_id = create_demo_run(
        run_name = run_name,
        experiment_id = experiment.experiment_id,
        run_tags = run_tags
    )
    print(f"Created run with id {run_id}")

## Search Run Method

```python
mlflow.search_runs(
    experiment_ids: Optional[List[str]] = None,
    filter_string: str = '',
    run_view_type: int = 1,
    max_results: int = 100000,
    order_by: Optional[List[str]] = None,
    output_format: str = 'pandas',
    search_all_experiments: bool = False, 
    experiment_names: Optional[List[str]] = None
) → Union[List[Run], pandas.DataFrame]
```

Search for Runs that fit the specified criteria.



In [ ]:
runs = mlflow.search_runs(experiment_names=["searching_for_runs"], search_all_experiments=False)

In [ ]:
runs.head()

In [ ]:
runs[runs["tags.run_type"] == "optimization"][["metrics.metric1","metrics.metric2","metrics.metric3","tags.run_type","tags.task"]]

## Using filter strings

In order to filter your MLflow runs, you will need to write search queries, which are pseudo-SQL conditions expressed in a distinct syntax.

In [ ]:
runs = mlflow.search_runs(experiment_names=["searching_for_runs"], filter_string="tags.run_type = 'optimization'")

In [ ]:
runs[["metrics.metric1","metrics.metric2","metrics.metric3","tags.run_type","tags.task"]]

In [ ]:
runs = mlflow.search_runs(experiment_names=["searching_for_runs"], filter_string="tags.run_type = 'optimization' AND metrics.metric1 > 0.8")

In [ ]:
runs[["metrics.metric1","metrics.metric2","metrics.metric3","tags.run_type","tags.task"]].head()

## Returning Run Objects

To return Run objects we have to specify this using the parameter `output_format`

**output_format** – The output format to be returned. If pandas, a pandas.DataFrame is returned and, if list, a list of mlflow.entities.Run is returned.

In [ ]:
runs = mlflow.search_runs(experiment_names=["searching_for_runs"], filter_string="tags.run_type = 'optimization'", output_format="list")

In [ ]:
for run in runs:
    print(type(run))
    print(f"Name: {run.info.run_name}, ID: {run.info.run_id}, Metrics: {run.data.metrics}")
    print("\n")